# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR,NuSVR
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import AgglomerativeClustering,KMeans
import random as rn
import math

# Loading data and removing rows with NaN values

In [2]:
df=pd.read_csv('C:\\Users\\abhic\\Modelling_in_Operation_Management\\2-Market\\finalDataset.csv')
df = df.replace(0, np.nan)
df.dropna(inplace=True)
df=df.iloc[:,1:]
df=df.reset_index()
df=df.drop(['index'],axis=1)
df.head()

,Open,High,Low,Close,Adj Close,Volume,MA5,MA10,MA20,DIF,MACD,KDJ.K,KDJ.D,PSYMA6,RSI6,RSI12,BIAS6,BIAS24
0,6.095000,6.120000,6.045000,6.085000,6.085000,526018.0000,6.044600,5.959400,6.292526,-0.031992,-0.043737,89.393939,83.310860,0.5,84.653465,53.188776,1.217044,-1.682733
1,9.117578,9.173124,9.058773,9.118762,9.118762,108976.8483,6.661352,6.284976,6.451814,0.198894,0.012312,98.393131,89.918095,0.5,98.173657,91.513651,39.069954,43.885515
2,6.090000,6.090000,5.980000,6.003000,6.003000,385568.0000,6.662352,6.293276,6.296026,0.145111,0.042067,6.295956,64.694342,0.5,49.746977,51.057923,-8.373904,-5.514342
3,6.145000,6.245000,6.130000,6.150000,6.150000,629111.0000,6.679352,6.314776,6.147588,0.112380,0.057451,10.641053,38.443380,0.5,51.181312,52.078950,-6.491757,-3.485434
4,6.185000,6.185000,6.120000,6.120000,6.120000,185961.0000,6.695352,6.346276,6.147338,0.083226,0.062986,9.754298,8.897102,0.5,50.429921,51.454435,-7.077406,-4.151781


- # features

In [3]:
X=pd.DataFrame(df.drop(columns=['Adj Close'],axis=1))
X.head()

,Open,High,Low,Close,Volume,MA5,MA10,MA20,DIF,MACD,KDJ.K,KDJ.D,PSYMA6,RSI6,RSI12,BIAS6,BIAS24
0,6.095000,6.120000,6.045000,6.085000,526018.0000,6.044600,5.959400,6.292526,-0.031992,-0.043737,89.393939,83.310860,0.5,84.653465,53.188776,1.217044,-1.682733
1,9.117578,9.173124,9.058773,9.118762,108976.8483,6.661352,6.284976,6.451814,0.198894,0.012312,98.393131,89.918095,0.5,98.173657,91.513651,39.069954,43.885515
2,6.090000,6.090000,5.980000,6.003000,385568.0000,6.662352,6.293276,6.296026,0.145111,0.042067,6.295956,64.694342,0.5,49.746977,51.057923,-8.373904,-5.514342
3,6.145000,6.245000,6.130000,6.150000,629111.0000,6.679352,6.314776,6.147588,0.112380,0.057451,10.641053,38.443380,0.5,51.181312,52.078950,-6.491757,-3.485434
4,6.185000,6.185000,6.120000,6.120000,185961.0000,6.695352,6.346276,6.147338,0.083226,0.062986,9.754298,8.897102,0.5,50.429921,51.454435,-7.077406,-4.151781


- # target

In [4]:
y=df['Adj Close']
y.head()

0    6.085000
1    9.118762
2    6.003000
3    6.150000
4    6.120000
Name: Adj Close, dtype: float64

# Correlation analysis : remove columns with correlation > 0.9

In [5]:
corrmat = X.corr() 
columns = np.full((corrmat.shape[0],), True, dtype=bool)
for i in range(corrmat.shape[0]):
    for j in range(i+1, corrmat.shape[0]):
        if corrmat.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = X.columns[columns]
data = X[selected_columns]
data.head()

,Open,Volume,DIF,KDJ.K,KDJ.D,PSYMA6,RSI6,RSI12,BIAS6,BIAS24
0,6.095000,526018.0000,-0.031992,89.393939,83.310860,0.5,84.653465,53.188776,1.217044,-1.682733
1,9.117578,108976.8483,0.198894,98.393131,89.918095,0.5,98.173657,91.513651,39.069954,43.885515
2,6.090000,385568.0000,0.145111,6.295956,64.694342,0.5,49.746977,51.057923,-8.373904,-5.514342
3,6.145000,629111.0000,0.112380,10.641053,38.443380,0.5,51.181312,52.078950,-6.491757,-3.485434
4,6.185000,185961.0000,0.083226,9.754298,8.897102,0.5,50.429921,51.454435,-7.077406,-4.151781


# PCA of data after correlation analysis

In [6]:
data_sc= StandardScaler().fit_transform(data)
pca = PCA(n_components=5)
pc = pca.fit_transform(data_sc)
pc_data = pd.DataFrame(data = pc
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(pc_data, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

# Brain Storm Optimization

In [13]:
def sigmoid(x):
    return (1/(1 + math.exp(-x)))

In [14]:
def svrCheck(X_train, y_train, X_val, y_val, sol):
    clf = NuSVR(kernel = 'linear', gamma = 'auto', C = sol[0], nu = sol[1])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    return (mean_squared_error(y_val, y_pred))

In [15]:
def randGenSol(n, d = 2):
    S = []
    for i in range(n):
        l = []
        l.append(rn.uniform(1,10))
        l.append(rn.random())
        S.append(l)
    #S = np.random.rand(n,2)
    return S

In [16]:
def clustProbGen(clus, n, m):
    clus = list(clus)
    uc = list(set(clus))
    p = [1/len(uc)]*len(uc)
    for i in range(m):
        p[i] = clus.count(i)/n
    return p

In [17]:
def probCheck(p):
    r = rn.random()
    if (r < p):
        return True
    return False

In [18]:
def stepFun(t, T, k = 1):
    x = ((0.5 * T- t)/k)
    res = sigmoid(x) * rn.uniform(0,1)
    return res

In [19]:
def genNewSol(x, t, T):
    n = len(x)
    y = [0]*n
    for i in range(n):
        y[i] = x[i] + stepFun(t, T) * rn.uniform(0,1)
    if(y[1] > 1):
        y[1] = sigmoid(y[1])
    return y

In [20]:
def combineTwoSol(x1, x2):
    n = len(x1)
    x = [0]*n
    r = rn.random()
    for i in range(n):
        x[i] = (r * x1[i]) + ((1-r) * x2[i])
    return x

In [21]:
def selClustCenters(X_train, y_train, X_val, y_val, S, lab, m):
    err = []
    cC = [[0,0]]*m
    cE = [9999999]*m
    best = 0
    for i in S:
            err.append(svrCheck(X_train, y_train, X_val, y_val, i))
    j = 0
    for i in lab:
        if(err[j] < cE[i]):
            cE[i] = err[j]
            cC[i] = j
        if(cE[i] < cE[best]):
            best = i
        j += 1
    return cC, best

In [22]:
def bso(X_train, y_train, X_val, y_val, n, m):
    pClustering = 0.5
    pGeneration = 0.5
    pOneCluster = 0.5
    pTwoCLuster = 0.5
    if(m == 1):
        pGeneration = 1
    
    T = 10 # max iterations
    
    Solutions = randGenSol(n)
    for t in range(T):
        clust = KMeans(n_clusters = m, random_state = 0).fit(Solutions)
        #clust = AgglomerativeClustering(n_clusters = m).fit(Solutions)
        prob = clustProbGen(clust.labels_, n, m)
        #print(clust.labels_)
        cCenters, best = selClustCenters(X_train, y_train, X_val, y_val, Solutions, clust.labels_, m)
        #print(svrCheck(X_train, y_train, X_val, y_val, Solutions[cCenters[best]]))
        if(probCheck(pClustering)):
            index = rn.choice(cCenters)
            new = randGenSol(1)[0]
            Solutions[index] = new
        newSols = []
        for i in range(n):
            flag = 0
            if(probCheck(pGeneration)):
                flag = 1
                selCluster = rn.choices(range(m), prob)[0]
                if(probCheck(pOneCluster)):
                    #print("Case-1")
                    index = cCenters[selCluster]
                    new = genNewSol(np.array(Solutions)[index], t, T)
                else:
                    #print("Case-2")
                    sel = list(rn.choice(np.array(Solutions)[clust.labels_ == selCluster]))
                    new = genNewSol(sel, t, T)
                    index = Solutions.index(sel)
            else:
                flag = 2
                selCluster1, selCluster2 = rn.choices(range(m), prob, k = 2)
                if(probCheck(pTwoCLuster)):
                    #print("Case-3")
                    index1 = cCenters[selCluster1]
                    index2 = cCenters[selCluster2]
                    comb = combineTwoSol(np.array(Solutions)[index1], Solutions[index2])
                    new = genNewSol(comb, t, T)
                else:
                    #print("Case-4")
                    sel1 = list(rn.choice(np.array(Solutions)[clust.labels_ == selCluster1]))
                    sel2 = list(rn.choice(np.array(Solutions)[clust.labels_ == selCluster2]))
                    comb = combineTwoSol(sel1, sel2)
                    new = genNewSol(comb, t, T)
                    index1 = Solutions.index(sel1)
                    index2 = Solutions.index(sel2)
            if(flag == 1):
                if(svrCheck(X_train, y_train, X_val, y_val, new) < svrCheck(X_train, y_train, X_val, y_val, Solutions[index])):
                    Solutions[index] = new
            elif(flag == 2):
                if(svrCheck(X_train, y_train, X_val, y_val, new) < svrCheck(X_train, y_train, X_val, y_val, Solutions[index1])):
                    Solutions[index1] = new
                elif(svrCheck(X_train, y_train, X_val, y_val, new) < svrCheck(X_train, y_train, X_val, y_val, Solutions[index2])):
                    Solutions[index2] = new 
    
    cCenters, best = selClustCenters(X_train, y_train, X_val, y_val, Solutions, clust.labels_, m)
    print("Validation MSE:", svrCheck(X_train, y_train, X_val, y_val, Solutions[cCenters[best]]))
    return Solutions[cCenters[best]]

# NuSVR models

In [38]:
reg_l = NuSVR(gamma='auto')
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.822147376508133
MSE : 0.2979505477464509


In [37]:
def checkK(n):
    k = list(range(1,n))
    E = []
    for i in k:
        best = bso(X_train, y_train, X_val, y_val, 10, i)
        print("Parameters :", best)
        E.append(svrCheck(X_train, y_train, X_test, y_test, best))
        print("Test MSE:", E[i-1])
    plt.plot(k,E)
#for i in range(10):
    #checkK(6)

In [67]:
best = bso(X_train, y_train, X_val, y_val, 10, 3)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.0669800864859801
Parameters : [6.376130237847938, 0.9030244337169016]
Test MSE: 0.1048452951569799


In [72]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear', C = best[0], nu = best[1])
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9374227723055467
MSE : 0.10483353521557319


In [71]:
best = bso(X_train, y_train, X_val, y_val, 10, 3)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.06696769362593122
Parameters : [7.530235833029731, 0.9088633624406102]
Test MSE: 0.10483353521557319


In [87]:
# svr with linear kernal
reg_l=SVR(kernel='linear')
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9375086140560136
MSE : 0.10468972740397643


In [61]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear')
gridsearchcv = GridSearchCV(reg_l, {'C':[0, 10], 'nu':[0,1]})
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9382871855799708
MSE : 0.10338541258720095


In [73]:
best = bso(X_train, y_train, X_val, y_val, 100, 10)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.06690494804300366
Parameters : [6.761216940704663, 0.8703308301588357]
Test MSE: 0.10423706409986547


In [74]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear', C = best[0], nu = best[1])
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9377788178089638
MSE : 0.10423706409986547


In [75]:
best = bso(X_train, y_train, X_val, y_val, 100, 8)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.06695494172767205
Parameters : [5.82667970562841, 0.9062638174511335]
Test MSE: 0.10481781064128536


In [76]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear', C = best[0], nu = best[1])
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9374321586175087
MSE : 0.10481781064128536


In [24]:
best = bso(X_train, y_train, X_val, y_val, 100, 5)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.06690096945448096
Parameters : [5.862091352846379, 0.8704365799098428]
Test MSE: 0.10422148750842707


In [25]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear', C = best[0], nu = best[1])
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9377881157870118
MSE : 0.10422148750842707


In [26]:
best = bso(X_train, y_train, X_val, y_val, 20, 5)
print("Parameters :", best)
print("Test MSE:", svrCheck(X_train, y_train, X_test, y_test, best))

Validation MSE: 0.06695956245358203
Parameters : [6.5229372321284895, 0.9123245773592755]
Test MSE: 0.1048658747005102


In [27]:
# svr with linear kernal
reg_l=NuSVR(kernel='linear', C = best[0], nu = best[1])
reg_l.fit(X_train,y_train)
p_l=reg_l.predict(X_test)
print('Regressor Score :',reg_l.score(X_test,y_test))
m_l=mse(y_test, p_l)
print('MSE :',m_l)

Regressor Score : 0.9374034682220941
MSE : 0.1048658747005102
